## Fake News Classifier Using LSTM

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('/content/gdrive/MyDrive/train.csv')

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
##Drop Nan Values
df = df.dropna()

In [11]:
## Get the Independent Features

X = df.drop('label', axis=1)

In [12]:
# Get the dependent features
y = df['label']

In [13]:
X.shape

(18285, 4)

In [10]:
y.shape

(18285,)

In [14]:
import tensorflow as tf

In [15]:
tf.__version__

'2.8.0'

In [16]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding 
from keras.layers import LSTM

In [17]:
### Vocabulary size
voc_size = 5000

## OneHot Representation

In [18]:
messages = X.copy()

In [19]:
messages.reset_index(inplace=True)

In [20]:
import nltk
import re
from nltk.corpus import stopwords

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [22]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [23]:
## Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus =[]
for i in range (len(messages)):
  review = re.sub('[^a-zA-Z]',' ', messages['title'][i]) #except a-zA-Z remaing all charcters are replace with space
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  review = ' '.join(review)
  corpus.append(review)

In [24]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [25]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[250, 8, 3278, 2724, 1663, 4366, 1924, 1856, 3937, 664],
 [972, 3984, 4005, 3256, 2623, 3383, 1047],
 [3330, 637, 4823, 4543],
 [1998, 4260, 3572, 2611, 4748, 2488],
 [1268, 2623, 4875, 1029, 838, 3695, 2623, 2767, 4090, 2267],
 [1644,
  1750,
  2594,
  3217,
  1481,
  4523,
  1764,
  2319,
  2797,
  4882,
  3556,
  3178,
  595,
  1019,
  1047],
 [4963, 3646, 4886, 1696, 243, 2280, 1577, 3004, 2367, 4529, 520],
 [2031, 4582, 867, 4437, 1356, 1152, 4523, 2568, 2367, 4529, 520],
 [1727, 4035, 3540, 3978, 923, 40, 2603, 320, 4523, 2749],
 [191, 3295, 4497, 1238, 1437, 3147, 881, 2246],
 [405, 1638, 534, 1709, 142, 4049, 2568, 4550, 971, 2358, 3037],
 [2611, 2604, 1663, 40, 4523, 1356],
 [3548, 1491, 4865, 3114, 510, 4181, 4302, 271, 3263],
 [1692, 2424, 507, 4735, 4588, 4521, 2154, 2367, 4529, 520],
 [690, 215, 1650, 4637, 4725, 2367, 4529, 520],
 [2445, 1518, 1936, 2457, 1031, 536, 277, 2570, 476, 2724],
 [63, 1755, 3984],
 [287, 1658, 483, 2271, 4523, 3296, 1072, 1047],
 [613, 1857, 40

## Embedding Representation

In [27]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen = sent_length)
print(embedded_docs)

[[   0    0    0 ... 1856 3937  664]
 [   0    0    0 ... 2623 3383 1047]
 [   0    0    0 ...  637 4823 4543]
 ...
 [   0    0    0 ... 2367 4529  520]
 [   0    0    0 ... 1103 1884 3251]
 [   0    0    0 ... 2238 1545 2899]]


In [28]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  250,
          8, 3278, 2724, 1663, 4366, 1924, 1856, 3937,  664], dtype=int32)

In [29]:
# Creating model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100)) #one LSTM layer with 100 neuron
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
len(embedded_docs), y.shape

(18285, (18285,))

In [31]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [36]:
X_final

array([[   0,    0,    0, ..., 1856, 3937,  664],
       [   0,    0,    0, ..., 2623, 3383, 1047],
       [   0,    0,    0, ...,  637, 4823, 4543],
       ...,
       [   0,    0,    0, ..., 2367, 4529,  520],
       [   0,    0,    0, ..., 1103, 1884, 3251],
       [   0,    0,    0, ..., 2238, 1545, 2899]], dtype=int32)

In [32]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [43]:
y_test

array([1, 0, 0, ..., 0, 1, 1])

## Model Training

In [37]:
## Finally Training
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 37ms/step - loss: 0.3483 - accuracy: 0.8360 - val_loss: 0.1968 - val_accuracy: 0.9163
Epoch 2/10
192/192 [==============================] - 7s 34ms/step - loss: 0.1424 - accuracy: 0.9441 - val_loss: 0.2106 - val_accuracy: 0.9221
Epoch 3/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0975 - accuracy: 0.9635 - val_loss: 0.2541 - val_accuracy: 0.9213
Epoch 4/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0691 - accuracy: 0.9764 - val_loss: 0.2497 - val_accuracy: 0.9208
Epoch 5/10
192/192 [==============================] - 6s 34ms/step - loss: 0.0435 - accuracy: 0.9854 - val_loss: 0.2808 - val_accuracy: 0.9142
Epoch 6/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0269 - accuracy: 0.9918 - val_loss: 0.3215 - val_accuracy: 0.9157
Epoch 7/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0129 - accuracy: 0.9964 - val_loss: 0.4883 - val_accuracy: 0.908

### Performance Metrics ANd Accuracy

In [49]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

In [50]:
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]], dtype=int32)

In [51]:
from sklearn.metrics import confusion_matrix

In [52]:
confusion_matrix(y_test, y_pred)

array([[3165,  254],
       [ 324, 2292]])

In [53]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9042253521126761

## Adding Dropout layer

In [55]:
from tensorflow.keras.layers import Dropout
# Creating model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100)) #one LSTM layer with 100 neuron
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)